In [1]:
import numpy as np

In [2]:
def txt2boxes():
    # 打开包含边界框信息的文本文件
    f = open(filename, 'r')
    dataSet = []  # 初始化存储边界框尺寸的列表
    for line in f:  # 遍历文件的每一行
        infos = line.split(" ")  # 通过空格分割每行的信息
        length = len(infos)
        for i in range(1, length):  # 从第二个元素开始遍历（跳过图像路径）
            # 计算边界框的宽度：xmax - xmin
            width = int(infos[i].split(",")[2]) - int(infos[i].split(",")[0])
            # 计算边界框的高度：ymax - ymin
            height = int(infos[i].split(",")[3]) - int(infos[i].split(",")[1])
            dataSet.append([width, height])  # 添加到数据集列表
    result = np.array(dataSet)  # 将列表转换为NumPy数组
    f.close()  # 关闭文件
    return result  # 返回包含所有边界框尺寸的NumPy数组

In [3]:
def iou(boxes, clusters):  # 计算一组边界框和聚类（锚框）的IoU
    n = boxes.shape[0]  # 边界框数量
    k = cluster_number  # 聚类数量

    # 计算边界框的面积
    box_area = boxes[:, 0] * boxes[:, 1]  # 宽*高
    box_area = box_area.repeat(k)  # 每个边界框面积重复k次
    box_area = np.reshape(box_area, (n, k))  # 重塑为n*k的矩阵

    # 计算聚类的面积
    cluster_area = clusters[:, 0] * clusters[:, 1]  # 聚类的宽*高
    cluster_area = np.tile(cluster_area, [1, n])  # 每个聚类面积重复n次
    cluster_area = np.reshape(cluster_area, (n, k))  # 重塑为n*k的矩阵

    # 计算宽度的最小值矩阵
    box_w_matrix = np.reshape(boxes[:, 0].repeat(k), (n, k))  # 边界框宽度重复k次并重塑
    cluster_w_matrix = np.reshape(np.tile(clusters[:, 0], (1, n)), (n, k))  # 聚类宽度重复n次并重塑
    min_w_matrix = np.minimum(cluster_w_matrix, box_w_matrix)  # 宽度的最小值

    # 计算高度的最小值矩阵
    box_h_matrix = np.reshape(boxes[:, 1].repeat(k), (n, k))  # 边界框高度重复k次并重塑
    cluster_h_matrix = np.reshape(np.tile(clusters[:, 1], (1, n)), (n, k))  # 聚类高度重复n次并重塑
    min_h_matrix = np.minimum(cluster_h_matrix, box_h_matrix)  # 高度的最小值

    # 计算交叉区域的面积
    inter_area = np.multiply(min_w_matrix, min_h_matrix)  # 交叉区域面积

    # 计算IoU
    result = inter_area / (box_area + cluster_area - inter_area)  # IoU计算公式
    return result  # 返回IoU值矩阵

In [4]:
def kmeans(boxes, k, dist=np.median):
    box_number = boxes.shape[0]  # 边界框的总数
    distances = np.empty((box_number, k))  # 存储每个边界框到聚类中心的距离
    last_nearest = np.zeros((box_number,))  # 上一次每个边界框最近的聚类中心
    np.random.seed()  # 初始化随机种子
    clusters = boxes[np.random.choice(box_number, k, replace=False)]  # 随机初始化k个聚类中心

    while True:
        distances = 1 - iou(boxes, clusters)  # 计算所有边界框与聚类中心的1-IoU作为距离

        current_nearest = np.argmin(distances, axis=1)  # 找到每个边界框最近的聚类中心
        if (last_nearest == current_nearest).all():  # 如果聚类中心不再变化，则结束迭代
            break
        for cluster in range(k):  # 更新每个聚类中心
            clusters[cluster] = dist(boxes[current_nearest == cluster], axis=0)

        last_nearest = current_nearest  # 更新上一次的聚类中心为当前聚类中心

    return clusters  # 返回聚类中心，即最终确定的锚框尺寸

In [5]:
def avg_iou(boxes, clusters):
    # 计算boxes与clusters之间的最大IoU值，然后计算这些最大值的平均值
    accuracy = np.mean([np.max(iou(boxes, clusters), axis=1)])
    return accuracy  # 返回平均IoU值，作为聚类性能的衡量指标

In [6]:
# 设置聚类的数量和边界框信息文件的名称
cluster_number = 9
filename = "2007_trainval.txt"

# 从文本文件中提取所有边界框的尺寸
all_boxes = txt2boxes()

# 使用k-means算法对边界框的尺寸进行聚类，找到最佳的锚框尺寸
result = kmeans(all_boxes, cluster_number)

# 对聚类得到的锚框尺寸进行排序
result = result[np.lexsort(result.T[0, None])]

# 打印聚类得到的锚框尺寸
print("K anchors:\n {}".format(result))

# 计算并打印所有边界框尺寸与聚类结果之间的平均IoU值
print("Accuracy: {:.2f}%".format(avg_iou(all_boxes, result) * 100))

K anchors:
 [[ 25  36]
 [ 34  83]
 [ 62  50]
 [ 65 131]
 [118 209]
 [120  95]
 [209 290]
 [242 144]
 [385 295]]
Accuracy: 68.86%
